# 🏀 バスケットボール選手判別チャレンジ - EDA

このノートブックでは、データの理解を深めるための探索的データ分析を行います。

## 目次
1. データ読み込み
2. メタデータの基本統計
3. 画像ファイルの確認
4. 選手IDの分布
5. 時系列分析（クオーター、セッション、フレーム）
6. Bounding Boxの分析
7. 画角（angle）の比較
8. 画像の可視化
9. テストデータの特徴
10. まとめと考察

## 1. データ読み込み

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import matplotlib_fontja

import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
# plt.rcParams['font.family'] = ['Noto Sans CJK JP', 'Yu Gothic', 'Hiragino Sans', 'MS Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 6)

print("ライブラリの読み込み完了")


In [ ]:
# パス設定
DATA_DIR = Path("../data/raw")
IMAGE_DIR = DATA_DIR / "images"

In [ ]:
# メタデータ読み込み
train_meta = pd.read_csv(DATA_DIR / "train_meta.csv")
test_meta = pd.read_csv(DATA_DIR / "test_meta.csv")
test_top_meta = pd.read_csv(DATA_DIR / "test_top_meta.csv")
sample_submission = pd.read_csv(DATA_DIR / "sample_submission.csv")

print("データ読み込み完了")
print(f"train_meta: {train_meta.shape}")
print(f"test_meta: {test_meta.shape}")
print(f"test_top_meta: {test_top_meta.shape}")
print(f"sample_submission: {sample_submission.shape}")

## 2. メタデータの基本統計

In [ ]:
# 学習データの基本情報
print("=" * 60)
print("📊 学習データ（train_meta）の基本情報")
print("=" * 60)
print(train_meta.info())
print("\n最初の5行:")
display(train_meta.head())

print("\n基本統計量:")
display(train_meta.describe())

In [ ]:
# テストデータの基本情報
print("=" * 60)
print("📊 テストデータ（test_meta）の基本情報")
print("=" * 60)
print(test_meta.info())
print("\n最初の5行:")
display(test_meta.head())

print("\n基本統計量:")
display(test_meta.describe())

In [ ]:
# テスト補助データ（top視点）の基本情報
print("=" * 60)
print("📊 テスト補助データ（test_top_meta）の基本情報")
print("=" * 60)
print(test_top_meta.info())
print("\n最初の5行:")
display(test_top_meta.head())

## 3. 画像ファイルの確認

In [ ]:
# 画像ファイルの確認
if IMAGE_DIR.exists():
    image_files = list(IMAGE_DIR.glob("*.jpg"))
    print(f"📸 画像ファイル数: {len(image_files)}")
    
    # ファイル名の例を表示
    print("\n画像ファイル名の例:")
    for i, img_path in enumerate(image_files[:5]):
        print(f"  {i+1}. {img_path.name}")
    
    # ファイル名のパース例
    print("\nファイル名の構造:")
    sample_name = image_files[0].stem
    parts = sample_name.split("__")
    print(f"  サンプル: {sample_name}")
    print(f"  クオーター: {parts[0]}")
    print(f"  画角: {parts[1]}")
    print(f"  セッション: {parts[2]}")
    print(f"  フレーム: {parts[3]}")
else:
    print(f"⚠️ 画像ディレクトリが見つかりません: {IMAGE_DIR}")

In [ ]:
# 画像ファイルとメタデータの対応確認
def create_image_filename(row):
    """メタデータから画像ファイル名を生成"""
    return f"{row['quarter']}__{row['angle']}__{str(row['session']).zfill(2)}__{str(row['frame']).zfill(2)}.jpg"

train_meta['image_filename'] = train_meta.apply(create_image_filename, axis=1)
test_meta['image_filename'] = test_meta.apply(create_image_filename, axis=1)
test_top_meta['image_filename'] = test_top_meta.apply(create_image_filename, axis=1)

# 画像の存在確認（サンプル）
if IMAGE_DIR.exists():
    sample_images = train_meta['image_filename'].head(100).tolist()
    existing_count = sum([(IMAGE_DIR / img).exists() for img in sample_images])
    print(f"\n画像存在確認（最初の100件）: {existing_count}/{len(sample_images)} 存在")

## 4. 選手IDの分布

In [ ]:
# 選手IDのユニーク数と分布
print("=" * 60)
print("👥 選手ID分析")
print("=" * 60)

unique_labels = train_meta['label_id'].unique()
print(f"ユニークな選手ID数: {len(unique_labels)}")
print(f"選手ID一覧: {sorted(unique_labels)}")

# 選手IDの出現回数
label_counts = train_meta['label_id'].value_counts().sort_index()
print("\n選手IDごとの出現回数:")
display(label_counts)

In [ ]:
# 選手IDの分布を可視化
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 棒グラフ
axes[0].bar(label_counts.index, label_counts.values, color='steelblue', alpha=0.7)
axes[0].set_xlabel('選手ID', fontsize=12)
axes[0].set_ylabel('出現回数', fontsize=12)
axes[0].set_title('選手IDごとの出現回数', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# 円グラフ
axes[1].pie(label_counts.values, labels=label_counts.index, autopct='%1.1f%%', 
           startangle=90, colors=plt.cm.Set3.colors)
axes[1].set_title('選手IDの割合', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# 統計情報
print("\n選手IDの統計:")
print(f"  平均出現回数: {label_counts.mean():.2f}")
print(f"  標準偏差: {label_counts.std():.2f}")
print(f"  最小出現回数: {label_counts.min()}")
print(f"  最大出現回数: {label_counts.max()}")

## 5. 時系列分析（クオーター、セッション、フレーム）

In [ ]:
# クオーター分析
print("=" * 60)
print("⏱️ クオーター分析")
print("=" * 60)

quarter_train = train_meta['quarter'].value_counts().sort_index()
quarter_test = test_meta['quarter'].value_counts().sort_index()

print("学習データのクオーター分布:")
display(quarter_train)
print("\nテストデータのクオーター分布:")
display(quarter_test)

In [ ]:
# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 学習データ
axes[0].bar(quarter_train.index, quarter_train.values, color='skyblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('クオーター', fontsize=12)
axes[0].set_ylabel('データ数', fontsize=12)
axes[0].set_title('学習データ - クオーター分布', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# テストデータ
axes[1].bar(quarter_test.index, quarter_test.values, color='lightcoral', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('クオーター', fontsize=12)
axes[1].set_ylabel('データ数', fontsize=12)
axes[1].set_title('テストデータ - クオーター分布', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# セッション分析
print("=" * 60)
print("📹 セッション分析")
print("=" * 60)

session_train = train_meta['session'].value_counts().sort_index()
session_test = test_meta['session'].value_counts().sort_index()

print(f"学習データのセッション数: {train_meta['session'].nunique()}")
print(f"学習データのセッション値: {sorted(train_meta['session'].unique())}")
print(f"\nテストデータのセッション数: {test_meta['session'].nunique()}")
print(f"テストデータのセッション範囲: {test_meta['session'].min()} - {test_meta['session'].max()}")

In [ ]:
# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 学習データ
axes[0].bar(session_train.index, session_train.values, color='mediumseagreen', alpha=0.7)
axes[0].set_xlabel('セッション', fontsize=12)
axes[0].set_ylabel('データ数', fontsize=12)
axes[0].set_title('学習データ - セッション分布', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# テストデータ
axes[1].bar(session_test.index, session_test.values, color='orange', alpha=0.7)
axes[1].set_xlabel('セッション', fontsize=12)
axes[1].set_ylabel('データ数', fontsize=12)
axes[1].set_title('テストデータ - セッション分布', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# フレーム分析
print("=" * 60)
print("🎬 フレーム分析")
print("=" * 60)

print(f"学習データのフレーム範囲: {train_meta['frame'].min()} - {train_meta['frame'].max()}")
print(f"テストデータのフレーム範囲: {test_meta['frame'].min()} - {test_meta['frame'].max()}")

# クオーター・セッションごとのフレーム数
print("\n学習データ - クオーター・セッションごとのフレーム数（最初の5つ）:")
frame_per_session = train_meta.groupby(['quarter', 'session'])['frame'].agg(['min', 'max', 'count'])
display(frame_per_session.head())

## 6. Bounding Boxの分析

In [ ]:
# Bounding Boxのサイズ分析
print("=" * 60)
print("📦 Bounding Box分析")
print("=" * 60)

# 学習データのBbox統計
print("学習データのBbox統計:")
bbox_stats_train = train_meta[['x', 'y', 'w', 'h']].describe()
display(bbox_stats_train)

# テストデータのBbox統計
print("\nテストデータのBbox統計:")
bbox_stats_test = test_meta[['x', 'y', 'w', 'h']].describe()
display(bbox_stats_test)

In [ ]:
# Bboxのサイズ分布を可視化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 幅（w）の分布
axes[0, 0].hist(train_meta['w'], bins=50, alpha=0.7, label='Train', color='blue', edgecolor='black')
axes[0, 0].hist(test_meta['w'], bins=50, alpha=0.5, label='Test', color='red', edgecolor='black')
axes[0, 0].set_xlabel('幅 (w)', fontsize=11)
axes[0, 0].set_ylabel('頻度', fontsize=11)
axes[0, 0].set_title('Bbox幅の分布', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 高さ（h）の分布
axes[0, 1].hist(train_meta['h'], bins=50, alpha=0.7, label='Train', color='blue', edgecolor='black')
axes[0, 1].hist(test_meta['h'], bins=50, alpha=0.5, label='Test', color='red', edgecolor='black')
axes[0, 1].set_xlabel('高さ (h)', fontsize=11)
axes[0, 1].set_ylabel('頻度', fontsize=11)
axes[0, 1].set_title('Bbox高さの分布', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# x座標の分布
axes[1, 0].hist(train_meta['x'], bins=50, alpha=0.7, label='Train', color='blue', edgecolor='black')
axes[1, 0].hist(test_meta['x'], bins=50, alpha=0.5, label='Test', color='red', edgecolor='black')
axes[1, 0].set_xlabel('X座標', fontsize=11)
axes[1, 0].set_ylabel('頻度', fontsize=11)
axes[1, 0].set_title('Bbox X座標の分布', fontsize=12, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# y座標の分布
axes[1, 1].hist(train_meta['y'], bins=50, alpha=0.7, label='Train', color='blue', edgecolor='black')
axes[1, 1].hist(test_meta['y'], bins=50, alpha=0.5, label='Test', color='red', edgecolor='black')
axes[1, 1].set_xlabel('Y座標', fontsize=11)
axes[1, 1].set_ylabel('頻度', fontsize=11)
axes[1, 1].set_title('Bbox Y座標の分布', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Bboxのアスペクト比
train_meta['aspect_ratio'] = train_meta['w'] / train_meta['h']
test_meta['aspect_ratio'] = test_meta['w'] / test_meta['h']

print("Bboxアスペクト比の統計:")
print(f"学習データ - 平均: {train_meta['aspect_ratio'].mean():.3f}, 中央値: {train_meta['aspect_ratio'].median():.3f}")
print(f"テストデータ - 平均: {test_meta['aspect_ratio'].mean():.3f}, 中央値: {test_meta['aspect_ratio'].median():.3f}")

plt.figure(figsize=(10, 5))
plt.hist(train_meta['aspect_ratio'], bins=50, alpha=0.7, label='Train', color='blue', edgecolor='black')
plt.hist(test_meta['aspect_ratio'], bins=50, alpha=0.5, label='Test', color='red', edgecolor='black')
plt.xlabel('アスペクト比 (w/h)', fontsize=12)
plt.ylabel('頻度', fontsize=12)
plt.title('Bboxアスペクト比の分布', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## 7. 画角（angle）の比較

In [ ]:
# 画角の分布
print("=" * 60)
print("📷 画角分析")
print("=" * 60)

angle_train = train_meta['angle'].value_counts()
angle_test = test_meta['angle'].value_counts()
angle_test_top = test_top_meta['angle'].value_counts()

print("学習データの画角分布:")
display(angle_train)
print("\nテストデータの画角分布:")
display(angle_test)
print("\nテスト補助データの画角分布:")
display(angle_test_top)

In [ ]:
# 画角ごとのBbox統計
print("\n画角ごとのBbox統計（学習データ）:")
for angle in train_meta['angle'].unique():
    print(f"\n{angle.upper()}:")
    angle_data = train_meta[train_meta['angle'] == angle]
    print(f"  データ数: {len(angle_data)}")
    print(f"  Bbox幅 - 平均: {angle_data['w'].mean():.2f}, 範囲: [{angle_data['w'].min()}, {angle_data['w'].max()}]")
    print(f"  Bbox高さ - 平均: {angle_data['h'].mean():.2f}, 範囲: [{angle_data['h'].min()}, {angle_data['h'].max()}]")

In [ ]:
# 画角ごとの選手ID分布
print("\n画角ごとの選手ID分布:")
for angle in train_meta['angle'].unique():
    angle_data = train_meta[train_meta['angle'] == angle]
    label_dist = angle_data['label_id'].value_counts().sort_index()
    print(f"\n{angle.upper()}:")
    display(label_dist)

In [ ]:
# 可視化
angle_label_pivot = train_meta.groupby(['angle', 'label_id']).size().unstack(fill_value=0)

fig, ax = plt.subplots(figsize=(12, 6))
angle_label_pivot.T.plot(kind='bar', ax=ax, color=['skyblue', 'lightcoral'], alpha=0.7, edgecolor='black')
ax.set_xlabel('選手ID', fontsize=12)
ax.set_ylabel('データ数', fontsize=12)
ax.set_title('画角ごとの選手ID分布', fontsize=14, fontweight='bold')
ax.legend(title='画角', fontsize=10)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 8. 画像の可視化

In [ ]:
# 画像サイズの確認
if IMAGE_DIR.exists() and len(list(IMAGE_DIR.glob("*.jpg"))) > 0:
    sample_image_path = list(IMAGE_DIR.glob("*.jpg"))[0]
    sample_img = Image.open(sample_image_path)
    
    print("=" * 60)
    print("🖼️ 画像情報")
    print("=" * 60)
    print(f"画像サイズ: {sample_img.size} (width x height)")
    print(f"画像モード: {sample_img.mode}")
    print(f"ファイルサイズ: {sample_image_path.stat().st_size / 1024:.2f} KB")

In [ ]:
import random
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from pathlib import Path

def visualize_side_top_pairs(meta_df, image_dir, n_samples=4):
    """
    sideの画像とそれに対応するtop画像をペアで可視化
    
    Parameters:
    -----------
    meta_df : pd.DataFrame or pl.DataFrame
        メタデータ
    image_dir : Path
        画像ディレクトリ
    n_samples : int
        表示するサンプル数
    """
    if not image_dir.exists():
        print(f"画像ディレクトリが見つかりません: {image_dir}")
        return
    
    # Polarsの場合はPandasに変換
    if hasattr(meta_df, 'to_pandas'):
        meta_df = meta_df.to_pandas()
    
    # sideの画像のみをフィルタ
    side_df = meta_df[meta_df['angle'] == 'side'].reset_index(drop=True)
    
    if len(side_df) == 0:
        print("sideの画像が見つかりません")
        return
    
    # ランダムにsideサンプルを選択
    sample_indices = random.sample(range(len(side_df)), min(n_samples, len(side_df)))
    
    # サブプロット作成（n_samples行 × 2列）
    fig, axes = plt.subplots(n_samples, 2, figsize=(12, 5 * n_samples))
    
    # axesが1次元の場合は2次元に変換
    if n_samples == 1:
        axes = axes.reshape(1, -1)
    
    for row_idx, sample_idx in enumerate(sample_indices):
        side_row = side_df.iloc[sample_idx]
        
        # === Side画像の処理 ===
        side_img_path = image_dir / side_row['image_filename']
        
        if side_img_path.exists():
            try:
                # Side画像読み込み
                side_img = Image.open(side_img_path)
                axes[row_idx, 0].imshow(side_img)
                
                # Bboxを描画
                rect = Rectangle(
                    (side_row['x'], side_row['y']), 
                    side_row['w'], side_row['h'],
                    linewidth=3, edgecolor='red', facecolor='none'
                )
                axes[row_idx, 0].add_patch(rect)
                
                # タイトル
                title = f"SIDE | Q{side_row['quarter']} S{side_row['session']} F{side_row['frame']}"
                if 'label_id' in side_row:
                    title += f"\nPlayer ID: {side_row['label_id']}"
                axes[row_idx, 0].set_title(title, fontsize=11, fontweight='bold', color='blue')
                
            except Exception as e:
                axes[row_idx, 0].text(0.5, 0.5, f"読み込みエラー\n{str(e)}", 
                                     ha='center', va='center', color='red')
        else:
            axes[row_idx, 0].text(0.5, 0.5, f"画像なし\n{side_row['image_filename']}", 
                                 ha='center', va='center', color='red')
        
        axes[row_idx, 0].axis('off')
        
        # === Top画像の処理 ===
        # sideのファイル名からtopのファイル名を生成
        side_filename = side_row['image_filename']
        # 例: "Q1_S1_side_F00001.jpg" -> "Q1_S1_top_F00001.jpg"
        top_filename = side_filename.replace('side', 'top')
        top_img_path = image_dir / top_filename
        
        # または、メタデータから同じquarter, session, frameのtop画像を検索
        top_row = meta_df[
            (meta_df['quarter'] == side_row['quarter']) &
            (meta_df['session'] == side_row['session']) &
            (meta_df['frame'] == side_row['frame']) &
            (meta_df['angle'] == 'top')
        ]
        
        if len(top_row) > 0:
            top_row = top_row.iloc[0]
            top_img_path = image_dir / top_row['image_filename']
            
            if top_img_path.exists():
                try:
                    # Top画像読み込み
                    top_img = Image.open(top_img_path)
                    axes[row_idx, 1].imshow(top_img)
                    
                    # Bboxを描画
                    rect = Rectangle(
                        (top_row['x'], top_row['y']), 
                        top_row['w'], top_row['h'],
                        linewidth=3, edgecolor='green', facecolor='none'
                    )
                    axes[row_idx, 1].add_patch(rect)
                    
                    # タイトル
                    title = f"TOP | Q{top_row['quarter']} S{top_row['session']} F{top_row['frame']}"
                    if 'label_id' in top_row:
                        title += f"\nPlayer ID: {top_row['label_id']}"
                    axes[row_idx, 1].set_title(title, fontsize=11, fontweight='bold', color='green')
                    
                except Exception as e:
                    axes[row_idx, 1].text(0.5, 0.5, f"読み込みエラー\n{str(e)}", 
                                         ha='center', va='center', color='red')
            else:
                axes[row_idx, 1].text(0.5, 0.5, f"TOP画像なし\n{top_filename}", 
                                     ha='center', va='center', color='orange', fontsize=9)
        else:
            axes[row_idx, 1].text(0.5, 0.5, 
                                 f"対応するTOP画像が\nメタデータに見つかりません", 
                                 ha='center', va='center', color='orange', fontsize=9)
        
        axes[row_idx, 1].axis('off')
    
    plt.suptitle('Side vs Top 画像比較', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    # 統計情報
    print("\n=== 統計情報 ===")
    print(f"総メタデータ数: {len(meta_df)}")
    print(f"Side画像数: {len(side_df)}")
    print(f"Top画像数: {len(meta_df[meta_df['angle'] == 'top'])}")
    print(f"表示サンプル数: {len(sample_indices)}")


def visualize_player_samples(train_meta, image_dir, label_id, n_samples=4):
    """
    特定の選手IDのside-topペア画像を表示
    **同じquarter, session, frameのside-top画像を確実にマッチング**
    
    Parameters:
    -----------
    train_meta : DataFrame
        メタデータ（sideとtopの両方を含む）
    image_dir : Path
        画像ディレクトリ
    label_id : int or str
        選手ID
    n_samples : int
        表示するサンプル数
    """
    # Polarsの場合はPandasに変換
    if hasattr(train_meta, 'to_pandas'):
        train_meta_pd = train_meta.to_pandas()
    else:
        train_meta_pd = train_meta
    
    # 特定の選手IDでフィルタ
    player_data = train_meta_pd[train_meta_pd['label_id'] == label_id]
    
    if len(player_data) == 0:
        print(f"選手ID {label_id} のデータが見つかりません")
        return
    
    # sideのデータのみ抽出
    side_data = player_data[player_data['angle'] == 'side'].reset_index(drop=True)
    
    if len(side_data) == 0:
        print(f"選手ID {label_id} のside画像が見つかりません")
        return
    
    print(f"\n{'='*50}")
    print(f"選手ID: {label_id}")
    print(f"総サンプル数: {len(player_data)}")
    print(f"Side画像数: {len(side_data)}")
    print(f"Top画像数: {len(player_data[player_data['angle'] == 'top'])}")
    print(f"{'='*50}\n")
    
    # ランダムにsideサンプルを選択
    sample_indices = random.sample(range(len(side_data)), min(n_samples, len(side_data)))
    
    # サブプロット作成
    fig, axes = plt.subplots(n_samples, 2, figsize=(12, 5 * n_samples))
    if n_samples == 1:
        axes = axes.reshape(1, -1)
    
    matched_count = 0
    
    for row_idx, sample_idx in enumerate(sample_indices):
        side_row = side_data.iloc[sample_idx]
        
        # === Side画像の処理 ===
        side_img_path = image_dir / side_row['image_filename']
        
        if side_img_path.exists():
            try:
                side_img = Image.open(side_img_path)
                axes[row_idx, 0].imshow(side_img)
                
                # Bbox描画
                rect = Rectangle(
                    (side_row['x'], side_row['y']),
                    side_row['w'], side_row['h'],
                    linewidth=3, edgecolor='red', facecolor='none'
                )
                axes[row_idx, 0].add_patch(rect)
                
                # タイトル
                title = f"SIDE | Q{side_row['quarter']} S{side_row['session']} F{side_row['frame']}"
                title += f"\nPlayer ID: {side_row['label_id']}"
                axes[row_idx, 0].set_title(title, fontsize=11, fontweight='bold', color='blue')
                
            except Exception as e:
                axes[row_idx, 0].text(0.5, 0.5, f"エラー\n{str(e)}", 
                                     ha='center', va='center', color='red')
        else:
            axes[row_idx, 0].text(0.5, 0.5, "画像なし", ha='center', va='center', color='red')
        
        axes[row_idx, 0].axis('off')
        
        # === 対応するTop画像の検索 ===
        # 同じquarter, session, frame, label_idのtop画像を検索
        top_row = player_data[
            (player_data['quarter'] == side_row['quarter']) &
            (player_data['session'] == side_row['session']) &
            (player_data['frame'] == side_row['frame']) &
            (player_data['label_id'] == side_row['label_id']) &
            (player_data['angle'] == 'top')
        ]
        
        if len(top_row) > 0:
            top_row = top_row.iloc[0]
            top_img_path = image_dir / top_row['image_filename']
            
            if top_img_path.exists():
                try:
                    top_img = Image.open(top_img_path)
                    axes[row_idx, 1].imshow(top_img)
                    
                    # Bbox描画
                    rect = Rectangle(
                        (top_row['x'], top_row['y']),
                        top_row['w'], top_row['h'],
                        linewidth=3, edgecolor='green', facecolor='none'
                    )
                    axes[row_idx, 1].add_patch(rect)
                    
                    # タイトル
                    title = f"TOP | Q{top_row['quarter']} S{top_row['session']} F{top_row['frame']}"
                    title += f"\nPlayer ID: {top_row['label_id']}"
                    axes[row_idx, 1].set_title(title, fontsize=11, fontweight='bold', color='green')
                    
                    matched_count += 1
                    
                    # マッチング確認（デバッグ用）
                    print(f"✓ マッチング成功 {row_idx+1}: Q{side_row['quarter']} S{side_row['session']} F{side_row['frame']}")
                    
                except Exception as e:
                    axes[row_idx, 1].text(0.5, 0.5, f"エラー\n{str(e)}", 
                                         ha='center', va='center', color='red')
            else:
                axes[row_idx, 1].text(0.5, 0.5, f"TOP画像なし\n{top_row['image_filename']}", 
                                     ha='center', va='center', color='orange')
        else:
            axes[row_idx, 1].text(0.5, 0.5, 
                                 f"対応するTOP画像なし\nQ{side_row['quarter']} S{side_row['session']} F{side_row['frame']}", 
                                 ha='center', va='center', color='orange', fontsize=9)
            print(f"✗ マッチング失敗 {row_idx+1}: Q{side_row['quarter']} S{side_row['session']} F{side_row['frame']}")
        
        axes[row_idx, 1].axis('off')
    
    plt.suptitle(f"Player ID: {label_id} - Side vs Top (マッチ率: {matched_count}/{n_samples})", 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    print(f"\n総マッチング成功: {matched_count}/{n_samples}")


# 使用例1: 全体からランダムにside-topペアを表示
print("=== 全体からランダムサンプル ===")
visualize_side_top_pairs(train_meta, IMAGE_DIR, n_samples=4)

# 使用例2: 特定の選手IDのside-topペアを表示
print("\n=== 特定選手のサンプル ===")
for label_id in sorted(train_meta['label_id'].unique())[:3]:  # 最初の3人
    visualize_player_samples(train_meta, IMAGE_DIR, label_id, n_samples=4)

## 9. テストデータの特徴

In [ ]:
# テストデータとテスト補助データの対応
print("=" * 60)
print("🔍 テストデータの特徴")
print("=" * 60)

# セッションごとのデータ数
print("テストデータのセッションごとのデータ数（上位10）:")
test_session_counts = test_meta.groupby(['quarter', 'session']).size().sort_values(ascending=False)
display(test_session_counts.head(10))

print("\nテスト補助データ（top）のセッション:")
test_top_sessions = test_top_meta.groupby(['quarter', 'session']).size()
display(test_top_sessions)

In [ ]:
# テストデータとテスト補助データの重なり確認
test_sessions = set(test_meta[['quarter', 'session']].apply(tuple, axis=1))
test_top_sessions = set(test_top_meta[['quarter', 'session']].apply(tuple, axis=1))

overlap = test_sessions & test_top_sessions
print(f"\nセッションの重なり:")
print(f"  test_metaのセッション数: {len(test_sessions)}")
print(f"  test_top_metaのセッション数: {len(test_top_sessions)}")
print(f"  重複セッション数: {len(overlap)}")
print(f"  test_metaのみのセッション数: {len(test_sessions - test_top_sessions)}")

In [ ]:
def visualize_test_side_top_pairs(
    side_meta, 
    top_meta, 
    image_dir, 
    n_samples=4,
    match_cols=['quarter', 'session', 'frame']
):
    """
    Test用: side_metaとtop_metaを別々に受け取ってペアで可視化
    
    Parameters:
    -----------
    side_meta : pd.DataFrame or pl.DataFrame
        sideのメタデータ
    top_meta : pd.DataFrame or pl.DataFrame
        topのメタデータ
    image_dir : Path
        画像ディレクトリ
    n_samples : int
        表示するサンプル数
    match_cols : list
        side-topをマッチングするための列
    """
    if not image_dir.exists():
        print(f"画像ディレクトリが見つかりません: {image_dir}")
        return
    
    # Polarsの場合はPandasに変換
    if hasattr(side_meta, 'to_pandas'):
        side_meta = side_meta.to_pandas()
    if hasattr(top_meta, 'to_pandas'):
        top_meta = top_meta.to_pandas()
    
    print(f"Side メタデータ: {len(side_meta)} 件")
    print(f"Top メタデータ: {len(top_meta)} 件")
    
    # ランダムにsideサンプルを選択
    sample_indices = random.sample(range(len(side_meta)), min(n_samples, len(side_meta)))
    
    # サブプロット作成（n_samples行 × 2列）
    fig, axes = plt.subplots(n_samples, 2, figsize=(12, 5 * n_samples))
    
    # axesが1次元の場合は2次元に変換
    if n_samples == 1:
        axes = axes.reshape(1, -1)
    
    matched_count = 0
    
    for row_idx, sample_idx in enumerate(sample_indices):
        side_row = side_meta.iloc[sample_idx]
        
        # === Side画像の処理 ===
        side_img_path = image_dir / side_row['image_filename']
        
        if side_img_path.exists():
            try:
                # Side画像読み込み
                side_img = Image.open(side_img_path)
                axes[row_idx, 0].imshow(side_img)
                
                # Bboxを描画
                rect = Rectangle(
                    (side_row['x'], side_row['y']), 
                    side_row['w'], side_row['h'],
                    linewidth=3, edgecolor='red', facecolor='none'
                )
                axes[row_idx, 0].add_patch(rect)
                
                # タイトル
                title = f"SIDE | Q{side_row['quarter']} S{side_row['session']} F{side_row['frame']}"
                if 'label_id' in side_row:
                    title += f"\nPlayer ID: {side_row['label_id']}"
                axes[row_idx, 0].set_title(title, fontsize=11, fontweight='bold', color='blue')
                
            except Exception as e:
                axes[row_idx, 0].text(0.5, 0.5, f"読み込みエラー\n{str(e)}", 
                                     ha='center', va='center', color='red')
        else:
            axes[row_idx, 0].text(0.5, 0.5, f"画像なし\n{side_row['image_filename']}", 
                                 ha='center', va='center', color='red')
        
        axes[row_idx, 0].axis('off')
        
        # === Top画像の検索とマッチング ===
        # match_colsに基づいて対応するtop画像を検索
        condition = True
        for col in match_cols:
            if col in side_row and col in top_meta.columns:
                condition = condition & (top_meta[col] == side_row[col])
        
        matched_top = top_meta[condition]
        
        if len(matched_top) > 0:
            top_row = matched_top.iloc[0]
            top_img_path = image_dir / top_row['image_filename']
            
            if top_img_path.exists():
                try:
                    # Top画像読み込み
                    top_img = Image.open(top_img_path)
                    axes[row_idx, 1].imshow(top_img)
                    
                    # Bboxを描画
                    rect = Rectangle(
                        (top_row['x'], top_row['y']), 
                        top_row['w'], top_row['h'],
                        linewidth=3, edgecolor='green', facecolor='none'
                    )
                    axes[row_idx, 1].add_patch(rect)
                    
                    # タイトル
                    title = f"TOP | Q{top_row['quarter']} S{top_row['session']} F{top_row['frame']}"
                    if 'label_id' in top_row:
                        title += f"\nPlayer ID: {top_row['label_id']}"
                    axes[row_idx, 1].set_title(title, fontsize=11, fontweight='bold', color='green')
                    
                    matched_count += 1
                    
                except Exception as e:
                    axes[row_idx, 1].text(0.5, 0.5, f"読み込みエラー\n{str(e)}", 
                                         ha='center', va='center', color='red')
            else:
                axes[row_idx, 1].text(0.5, 0.5, f"TOP画像なし\n{top_row['image_filename']}", 
                                     ha='center', va='center', color='orange', fontsize=9)
        else:
            # マッチング情報を表示
            match_info = ", ".join([f"{col}={side_row[col]}" for col in match_cols if col in side_row])
            axes[row_idx, 1].text(0.5, 0.5, 
                                 f"対応するTOP画像なし\n{match_info}", 
                                 ha='center', va='center', color='orange', fontsize=9)
        
        axes[row_idx, 1].axis('off')
    
    plt.suptitle(f'Test Data: Side vs Top 画像比較 (マッチ率: {matched_count}/{n_samples})', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    # 統計情報
    print("\n=== マッチング統計 ===")
    print(f"Side サンプル数: {n_samples}")
    print(f"マッチング成功: {matched_count}/{n_samples} ({matched_count/n_samples*100:.1f}%)")
    
    # マッチングキーごとの統計
    print(f"\n=== マッチングキー: {match_cols} ===")
    for col in match_cols:
        if col in side_meta.columns and col in top_meta.columns:
            side_unique = set(side_meta[col].unique())
            top_unique = set(top_meta[col].unique())
            common = side_unique & top_unique
            print(f"{col}: Side={len(side_unique)}, Top={len(top_unique)}, 共通={len(common)}")


def visualize_test_player_samples(
    side_meta, 
    top_meta, 
    image_dir, 
    label_id=None, 
    n_samples=4
):
    """
    Test用: 特定の選手IDのside-topペアを表示
    
    Parameters:
    -----------
    side_meta : DataFrame
        sideのメタデータ
    top_meta : DataFrame
        topのメタデータ
    image_dir : Path
        画像ディレクトリ
    label_id : int or None
        選手ID（Noneの場合はランダム）
    n_samples : int
        表示するサンプル数
    """
    # Polarsの場合はPandasに変換
    if hasattr(side_meta, 'to_pandas'):
        side_meta = side_meta.to_pandas()
    if hasattr(top_meta, 'to_pandas'):
        top_meta = top_meta.to_pandas()
    
    # 特定の選手IDでフィルタ（指定された場合）
    if label_id is not None:
        if 'label_id' not in side_meta.columns:
            print("エラー: side_metaにlabel_id列がありません")
            return
        
        side_filtered = side_meta[side_meta['label_id'] == label_id]
        top_filtered = top_meta[top_meta['label_id'] == label_id] if 'label_id' in top_meta.columns else top_meta
        
        if len(side_filtered) == 0:
            print(f"選手ID {label_id} のデータが見つかりません")
            return
        
        print(f"\n{'='*50}")
        print(f"選手ID: {label_id}")
        print(f"Side サンプル数: {len(side_filtered)}")
        if 'label_id' in top_meta.columns:
            print(f"Top サンプル数: {len(top_filtered)}")
        print(f"{'='*50}\n")
    else:
        side_filtered = side_meta
        top_filtered = top_meta
        print("ランダムサンプルを表示します")
    
    # side-topペアで可視化
    visualize_test_side_top_pairs(side_filtered, top_filtered, image_dir, n_samples=n_samples)

In [ ]:
# テストデータの画像サンプル
# 例1: Test全体からランダムにside-topペアを表示
print("=== Testデータ: ランダムサンプル ===")
visualize_test_side_top_pairs(test_meta, test_top_meta, IMAGE_DIR, n_samples=4)

# # 例2: 特定の選手IDのside-topペアを表示（label_idがある場合）
# if 'label_id' in test_meta.columns:
#     print("\n=== Testデータ: 特定選手のサンプル ===")
#     unique_ids = test_meta['label_id'].unique()
#     for label_id in sorted(unique_ids)[:3]:  # 最初の3人
#         visualize_test_player_samples(test_meta, test_top_meta, IMAGE_DIR, label_id=label_id, n_samples=2)

# # 例3: 特定のクオーターのみ表示
# print("\n=== Testデータ: Q4のみ ===")
# q4_side = test_meta[test_meta['quarter'] == 'Q4']
# q4_top = test_top_meta[test_top_meta['quarter'] == 'Q4']
# visualize_test_side_top_pairs(q4_side, q4_top, IMAGE_DIR, n_samples=4)

# # 例4: マッチングキーをカスタマイズ
# print("\n=== カスタムマッチング（quarterとframeのみ）===")
# visualize_test_side_top_pairs(
#     test_meta, 
#     test_top_meta, 
#     IMAGE_DIR, 
#     n_samples=4,
#     match_cols=['quarter', 'frame']  # sessionを除外
# )

## 10. まとめと考察

In [ ]:
print("=" * 80)
print("📝 EDAまとめ")
print("=" * 80)

summary = f"""
【データ概要】
- 学習データ: {len(train_meta):,} 件（{train_meta['label_id'].nunique()}人の選手）
- テストデータ: {len(test_meta):,} 件
- テスト補助データ: {len(test_top_meta):,} 件

【選手ID】
- ユニーク選手数: {train_meta['label_id'].nunique()} 人
- ID範囲: {train_meta['label_id'].min()} - {train_meta['label_id'].max()}
- 最多出現選手: ID {train_meta['label_id'].value_counts().idxmax()} ({train_meta['label_id'].value_counts().max()}回)
- 最少出現選手: ID {train_meta['label_id'].value_counts().idxmin()} ({train_meta['label_id'].value_counts().min()}回)

【時系列情報】
- クオーター数: 学習={train_meta['quarter'].nunique()}, テスト={test_meta['quarter'].nunique()}
- セッション数: 学習={train_meta['session'].nunique()}, テスト={test_meta['session'].nunique()}
- フレーム範囲: 学習=[{train_meta['frame'].min()}-{train_meta['frame'].max()}], テスト=[{test_meta['frame'].min()}-{test_meta['frame'].max()}]

【Bounding Box】
- 平均サイズ: 幅={train_meta['w'].mean():.1f}px, 高さ={train_meta['h'].mean():.1f}px
- アスペクト比: {train_meta['aspect_ratio'].mean():.3f}

【画角】
- 学習データ: side={len(train_meta[train_meta['angle']=='side'])}, top={len(train_meta[train_meta['angle']=='top'])}
- テストデータ: side={len(test_meta[test_meta['angle']=='side'])}, top={len(test_meta[test_meta['angle']=='top'])}

【重要な気づき】
1. 学習データはsession=0のみだが、テストデータは複数セッションが存在
2. テストデータは主にside画角のみ（一部セッションでtop画角も提供）
3. 選手IDの出現頻度に偏りがある可能性
4. 未知選手（-1）の検出が必要
"""

print(summary)

In [ ]:
# 追加の考察ポイント
print("\n【モデリングのヒント】")
print("""
✅ マルチビュー学習
   - side と top の画角を統合する手法が有効そう
   - Attention機構で画角間の対応付け

✅ 時系列情報の活用
   - 前後フレームの情報を利用した追跡
   - LSTM/Transformerで時間的一貫性を考慮

✅ 未知選手の検出
   - Out-of-Distribution検出手法
   - 信頼度スコアで-1を判定

✅ データ拡張
   - Bboxのサイズや位置のバリエーション
   - 画角変換、回転、スケーリング

✅ アンサンブル
   - 複数モデルの組み合わせで精度向上
   - side専用モデルとマルチビューモデルの統合
""")

print("\nEDA完了！ 🎉")

## 次のステップ
 
1. ベースラインモデルの構築（ResNet + bbox crop）
2. マルチビュー学習の検討
3. 時系列情報の活用
4. 未知選手検出の実装
5. アンサンブル手法の検討